In [1]:
MONGO_URI="<Insert here MongoDB Atlas connection URI>"


from pyspark.sql import SparkSession
from pyspark.sql.functions import col

#Avvia la Spark Session mappando con delle confguration input e output da  verso MongoDB
spark = SparkSession \
    .builder \
    .appName("myApp") \
    .getOrCreate()

In [2]:
#Lettura del file csv
syn_dataset=spark.read.option("header", "false").option("inferschema","true").csv("/FileStore/tables/syn_dos_11000.csv")

#Conversione esplicita delle colonne in Double
syn_dataset_double = syn_dataset.select(*(col(c).cast("Double").alias(c) for c in syn_dataset.columns[:-1]),col(syn_dataset.columns[-1])).withColumnRenamed("_c115","Class")

In [3]:
#Scrittura su MongoDB
syn_dataset_double.write.format("mongo").option("spark.mongodb.output.uri", MONGO_URI).option("database", "test").option("collection", "test.syn-dos_double").save()